In [1]:
#import unicodecsv as csv
import csv
#conda install unicodecsv

# Data Analysis Process

### Step 1: Get data

Categorical Data - Any value that if you ran mathematical operation on, you'd get nonsensical data

Ordinal Data - Any value that if you ran mathematical operation on it, you'd get sensical data

### Step 2: Clean Data

In [2]:
enrollment_data = []
file = open('./enrollments.csv', 'r')
data = csv.DictReader(file)
for i in data:
    enrollment_data.append(i)
file.close()

In [3]:
def read_from_csv(filepath):
    with open(filepath) as file:
        data = csv.DictReader(file)
        return list(data)

enrollment_data = read_from_csv('./enrollments.csv')
engagement_data = read_from_csv('./engagements.csv')
submission_data = read_from_csv('./submissions.csv')

In [4]:
from datetime import datetime as dt

In [6]:
def convert_to_date(string):
    if string == '':
        return None
    return dt.strptime(string, '%Y-%m-%d')
def convert_to_bool(string):
    if string == '':
        return None
    elif string == 'TRUE':
        return True
    elif string == 'FALSE':
        return False
def convert_to_int(string):
    if string == '':
        return None
    return int(string)    
def convert_to_float(string):
    if string == '':
        return None
    return float(string)


In [7]:
for r in enrollment_data:
    r['join_date'] = convert_to_date(r['join_date'])
    r['cancel_date'] = convert_to_date(r['cancel_date'])
    r['days_to_cancel'] = convert_to_int(r['days_to_cancel'])
    r['is_enrolled'] = convert_to_bool(r['is_enrolled'])
    r['is_cancelled'] = convert_to_bool(r['is_canceled'])

In [10]:
enrollment_data[0]

OrderedDict([('account_key', '700'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2017, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2017, 11, 16, 0, 0)),
             ('days_to_cancel', 6),
             ('is_enrolled', False),
             ('is_canceled', 'TRUE'),
             ('is_cancelled', True)])

In [8]:
for r in engagement_data:
    r['utc_date'] = convert_to_date(r['utc_date'])
    r['num_courses_visited'] = convert_to_int(r['num_courses_visited'])
    r['total_minutes_visited'] = convert_to_float(r['total_minutes_visited'])
    r['lessons_completed'] = convert_to_int(r['lessons_completed'])
    r['projects_completed'] = convert_to_int(r['projects_completed'])

In [11]:
engagement_data[0]

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2018, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])

In [9]:
for r in submission_data:
    r['creation_date'] = convert_to_date(r['creation_date'])
    r['completion_date'] = convert_to_date(r['completion_date'])

In [12]:
submission_data[0]

OrderedDict([('creation_date', datetime.datetime(2018, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2018, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

In [13]:
# Make consistent all column labels
# RUN ONLY ONCE
for r in engagement_data:
    r['account_key'] = r['acct']
    #destroy the old acct reference
    del r['acct']
# RUN ONLY ONCE

In [14]:
engagement_data[0]

OrderedDict([('utc_date', datetime.datetime(2018, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0),
             ('account_key', '0')])

In [18]:
def get_unique_records(dataset, column_name):
    unique_data = set()
    for row in dataset:
        unique_data.add(row[column_name])
    return unique_data

In [19]:
unique_enrollments = get_unique_records(enrollment_data, 'account_key')
unique_engagements = get_unique_records(engagement_data, 'account_key')
unique_submissions = get_unique_records(submission_data, 'account_key')

In [20]:
print(len(unique_enrollments))
print(len(unique_engagements))
print(len(unique_submissions))

1302
1237
743


In [21]:
outliers = 0
for r in enrollment_data:
    if r['account_key'] not in unique_engagements and r['join_date'] != r['cancel_date']:
        outliers +=1
        print(r, end= '\n\n')

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2018, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2018, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_enrolled', True), ('is_canceled', 'TRUE'), ('is_cancelled', True)])

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2018, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2018, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_enrolled', True), ('is_canceled', 'TRUE'), ('is_cancelled', True)])

OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2018, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_enrolled', True), ('is_canceled', 'FALSE'), ('is_cancelled', False)])



In [22]:
test_accounts = set()
for r in enrollment_data:
    if r['is_enrolled']:
        test_accounts.add(r['account_key'])

print(len(test_accounts))

6


In [23]:
test_accounts

{'1069', '1101', '1304', '312', '448', '818'}

In [32]:
#for any of the accounts found above, we need to remove their corresponding records from the other dataframe to prevent
#"screwing up" our data findings later down the road

def remove_test_accounts(dataset):
    records_to_remove = []
    for row_dict in dataset:
        if row_dict['account_key'] not in test_accounts:
            records_to_remove.append(row_dict)
    return records_to_remove

In [33]:
true_enrollments = remove_test_accounts(enrollment_data)
true_engagements = remove_test_accounts(engagement_data)
true_submissions = remove_test_accounts(submission_data)

In [34]:
print(len(true_enrollments))
print(len(true_engagements))
print(len(true_submissions))

1622
135656
3634


OrderedDict([('utc_date', datetime.datetime(2018, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0),
             ('account_key', '0')])

<h4>Find quality accounts in which:</h4>
<ol>
    <li>The account is currently active i.e. is_canceled is False</li>
    <li>days_to_cancel is greater than 7 i.e. the trial period</li>
    <li>The latest vald record for each particular VALID user</li>
</ol>

In [ ]:
# {
#     '488': datetime()
# }

In [36]:
accounts_in_good_standing = dict()
for row_dict in true_enrollments:
    #if the account is currently active
    if row_dict['days_to_cancel'] is not None and row_dict['cancel_date'] is not None:
        #if the difference in days_to_cancel and join_date is > the 7 day trial period
        if not row_dict['is_canceled'] or row_dict['days_to_cancel'] > 7:
            #grab most recent join date. We don't care about their previous accounts
            if row_dict['account_key'] not in accounts_in_good_standing or row_dict['join_date'] > accounts_in_good_standing[row_dict['account_key']]:
                #create a dictionary key with the account_key number and set its value to the most recent join_date
                accounts_in_good_standing[row_dict['account_key']] = row_dict['join_date']

In [37]:
accounts_in_good_standing

{'429': datetime.datetime(2018, 3, 10, 0, 0),
 '60': datetime.datetime(2018, 1, 14, 0, 0),
 '322': datetime.datetime(2018, 2, 12, 0, 0),
 '584': datetime.datetime(2018, 1, 14, 0, 0),
 '458': datetime.datetime(2017, 11, 10, 0, 0),
 '1058': datetime.datetime(2018, 1, 14, 0, 0),
 '45': datetime.datetime(2017, 11, 10, 0, 0),
 '315': datetime.datetime(2017, 11, 10, 0, 0),
 '408': datetime.datetime(2018, 4, 1, 0, 0),
 '51': datetime.datetime(2018, 3, 10, 0, 0),
 '323': datetime.datetime(2017, 11, 10, 0, 0),
 '130': datetime.datetime(2018, 4, 1, 0, 0),
 '550': datetime.datetime(2018, 5, 28, 0, 0),
 '44': datetime.datetime(2017, 11, 10, 0, 0),
 '440': datetime.datetime(2017, 11, 11, 0, 0),
 '57': datetime.datetime(2017, 11, 11, 0, 0),
 '1090': datetime.datetime(2017, 11, 11, 0, 0),
 '541': datetime.datetime(2017, 11, 12, 0, 0),
 '756': datetime.datetime(2017, 11, 15, 0, 0),
 '101': datetime.datetime(2017, 11, 12, 0, 0),
 '800': datetime.datetime(2018, 3, 4, 0, 0),
 '702': datetime.datetime(201

In [43]:
def remove_free_trials(dataset):
    free_trials_list = []
    for row_dict in dataset:
        if row_dict['account_key'] in accounts_in_good_standing:
            free_trials_list.append(row_dict)
    return free_trials_list

In [44]:
quality_enrollments = remove_free_trials(true_enrollments)
quality_engagements = remove_free_trials(true_engagements)
quality_submissions = remove_free_trials(true_submissions)

In [45]:
print(len(quality_enrollments))
print(len(quality_engagements))
print(len(quality_submissions))

719
55675
1110


### Step 3: Answer Questions with Data

In [46]:
# How many students were active on the website within the first week?

def engagements_within_first_week(date_join, date_of_first_engagement):
    time_delta = date_of_first_engagement - date_join
    if time_delta.days >= 0 and time_delta.days < 8:
        return True
    

for row_dict in quality_engagements:
    if row_dict['num_courses_visited'] > 0:
        row_dict['has_visited'] = 1
    else:
        row_dict['has_visited'] = 0

first_week_engagements = []
for row_dict in quality_engagements:
    if engagements_within_first_week(accounts_in_good_standing[row_dict['account_key']], row_dict['utc_date']):
        first_week_engagements.append(row_dict)

In [47]:
quality_engagements[0]

OrderedDict([('utc_date', datetime.datetime(2017, 11, 10, 0, 0)),
             ('num_courses_visited', 2),
             ('total_minutes_visited', 136.1835995),
             ('lessons_completed', 0),
             ('projects_completed', 0),
             ('account_key', '3'),
             ('has_visited', 1)])

In [48]:
print(len(first_week_engagements))

3559


In [54]:
# How many minutes did each student spend online in the first week?

from collections import defaultdict

def compare(dataset, dict_key):
    new_data = defaultdict(list)
    for row_dict in dataset:
        account_key = row_dict[dict_key]
        new_data[account_key].append(row_dict)
    return new_data

In [55]:
engagements_grouped_by_account = compare(first_week_engagements, 'account_key')

In [56]:
engagements_grouped_by_account['25']

[OrderedDict([('utc_date', datetime.datetime(2017, 11, 11, 0, 0)),
              ('num_courses_visited', 0),
              ('total_minutes_visited', 0.0),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '25'),
              ('has_visited', 0)]),
 OrderedDict([('utc_date', datetime.datetime(2017, 11, 12, 0, 0)),
              ('num_courses_visited', 0),
              ('total_minutes_visited', 0.0),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '25'),
              ('has_visited', 0)]),
 OrderedDict([('utc_date', datetime.datetime(2017, 11, 13, 0, 0)),
              ('num_courses_visited', 0),
              ('total_minutes_visited', 0.0),
              ('lessons_completed', 0),
              ('projects_completed', 0),
              ('account_key', '25'),
              ('has_visited', 0)]),
 OrderedDict([('utc_date', datetime.datetime(2017, 11, 14, 0, 0)),
      

In [57]:
# How many minutes did each student spend on the website total?

def sum_records(dataset, column):
    new_data = dict()
    for dict_key, dict_value in dataset.items():
        sum_ = 0
        for row in dict_value:
            sum_ += row[column]
        new_data[dict_key] = sum_
            
    return new_data 

In [58]:
minutes_per_accounts = sum_records(engagements_grouped_by_account, 'total_minutes_visited')

In [59]:
minutes_per_accounts

{'3': 1023.5579746999999,
 '5': 370.82494396,
 '15': 1071.12533757,
 '18': 155.1361575,
 '19': 0.0,
 '24': 943.1881172000001,
 '25': 0.0,
 '31': 247.27557963999996,
 '34': 138.1789523,
 '42': 174.30704300000002,
 '43': 88.822038,
 '44': 731.72366783,
 '45': 979.7454166629999,
 '51': 0.0,
 '52': 3.227523833,
 '53': 43.53851617,
 '57': 750.894147533,
 '60': 0.0,
 '62': 293.3831222,
 '64': 589.58194586,
 '65': 422.27366,
 '67': 1123.02533437,
 '69': 0.0,
 '72': 883.9906950330001,
 '74': 90.48065633,
 '76': 17.658779,
 '78': 6.609209333,
 '79': 416.495983833,
 '85': 500.49217067,
 '88': 106.16380783,
 '90': 4.842514,
 '91': 70.37317134,
 '92': 715.4277596699999,
 '93': 932.7172561699999,
 '97': 494.84799353,
 '98': 531.4097201300001,
 '99': 425.60350947,
 '101': 188.92038351,
 '105': 573.58079867,
 '106': 535.168407333,
 '108': 164.270890837,
 '110': 1403.143966737,
 '111': 169.1595922,
 '113': 139.57379017,
 '119': 103.90436599,
 '123': 0.0,
 '130': 0.0,
 '132': 1428.3456270000001,
 '134'

In [ ]:
# HOMEWORK:

# Take a look at data and come up with 3 good questions and use some of the methods done in this 
# Jupyter Notebook to successfully answer the questions.

In [100]:
#Question 1
#What is the number of completed lessons per unique engagement?

# def num_of_users(dataset):
lessons_sum = []
for row in quality_engagements:
    if row['lessons_completed']:
        lessons_sum.append(row['lessons_completed'])

    
print(sum(lessons_sum)/ len(unique_engagements))

4.90784155214228


In [107]:
#Question 2
# What percent of quality engagements completed at least 1 lesson?
bad_student_count = 0
good_student_count = 0
for row in quality_engagements:   
    if row['lessons_completed'] == 0:
        bad_student_count += 1
    elif row['lessons_completed'] >= 1:
        good_student_count += 1

print(good_student_count / (bad_student_count + good_student_count) *100 )
        

8.062864840592725


In [115]:
#Question 3
#What is the average number of days to cancel?

def average_cancel(dataset):
    status = []
    for row in dataset:
        if row['is_canceled'] == 'TRUE':
            status.append(row['days_to_cancel'])
    return status

num_true = average_cancel(true_enrollments)
average_days = (sum(num_true) / len(num_true)) / 2
print(average_days)

22.028776978417266
